# 신호등 인식

In [5]:
import cv2
import numpy as np

def detect_traffic_light(frame):
    """신호등 색상 감지 및 위치 반환"""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # RED Light
    lower_red1 = np.array([0, 120, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 120, 100])
    upper_red2 = np.array([180, 255, 255])

    # GREEN Light
    lower_green = np.array([40, 50, 50])
    upper_green = np.array([90, 255, 255])

    # YELLOW Light
    lower_yellow = np.array([15, 100, 100])
    upper_yellow = np.array([35, 255, 255])

    # Mask for each light
    mask_red = cv2.bitwise_or(
        cv2.inRange(hsv, lower_red1, upper_red1),
        cv2.inRange(hsv, lower_red2, upper_red2)
    )
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # detected light list
    detections = []

    for color, mask in [('RED', mask_red), ('GREEN', mask_green), ('YELLOW', mask_yellow)]:
        # remove noise
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 300:  # detecting range
                x, y, w, h = cv2.boundingRect(cnt)
                detections.append((color, (x, y, w, h)))

    return detections

def main():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Failed load Camera")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        detections = detect_traffic_light(frame)

        for color, (x, y, w, h) in detections:
            # 색상별 박스 색 지정
            box_color = (0, 0, 255) if color == 'RED' else (0, 255, 0) if color == 'GREEN' else (0, 255, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), box_color, 2)
            cv2.putText(frame, f"{color} LIGHT", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, box_color, 2)

        cv2.imshow("Traffic Light Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [4]:
import cv2
import numpy as np

def detect_traffic_light(frame):
    """신호등 색상 감지 및 위치 반환"""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # 색상 범위 정의
    lower_red1 = np.array([0, 120, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([160, 120, 100])
    upper_red2 = np.array([180, 255, 255])

    lower_green = np.array([40, 50, 50])
    upper_green = np.array([90, 255, 255])

    lower_yellow = np.array([15, 100, 100])
    upper_yellow = np.array([35, 255, 255])

    mask_red = cv2.bitwise_or(
        cv2.inRange(hsv, lower_red1, upper_red1),
        cv2.inRange(hsv, lower_red2, upper_red2)
    )
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    detections = []

    for color, mask in [('RED', mask_red), ('GREEN', mask_green), ('YELLOW', mask_yellow)]:
        # 노이즈 제거
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < 500 or area > 50000:
                continue  # 너무 작거나 큰 건 무시

            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = h / float(w)

            # 신호등은 보통 세로로 긴 형태 (예: 세로가 가로의 1.2배 이상)
            if aspect_ratio > 1.2 and aspect_ratio < 5.0:
                detections.append((color, (x, y, w, h)))

    return detections

def main():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("웹캠을 열 수 없습니다.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        detections = detect_traffic_light(frame)

        for color, (x, y, w, h) in detections:
            box_color = (0, 0, 255) if color == 'RED' else (0, 255, 0) if color == 'GREEN' else (0, 255, 255)
            cv2.rectangle(frame, (x, y), (x + w, y + h), box_color, 2)
            cv2.putText(frame, f"{color} LIGHT", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, box_color, 2)

        cv2.imshow("Traffic Light Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
